In [1]:
import pandas as pd
import numpy as np
from tkinter import ttk
from tkinter import filedialog
from tkinter import *
from tkinter.ttk import *
import os

In [2]:
# Inputs and Parameters

# File Path
fileName = "C:/Users/marym/OneDrive/Documents/PropellerCalc/propellers/PER3_20x10E.dat"
file = "PER3_20x10E"
#fileName = 'C:/Users/marym/OneDrive/Documents/PropellerCalc/propellers/' + file + '.dat'

# Constants
R = 8.3132                       # Gas Constant (J/K-mol)
M = 0.02896                      # Molar Mass of Earth's Air (kg)
g = 9.806                        # Force of Gravity (N)
P0 = 101325                      # Pressure at Sea Level (Pa) - Double Check Units with this value

# Starting Point
propDia = 20.0 * 0.0254          # Prop Diameter (in)
alt = 1289 * 0.3048              # Altitude w/ coversion (m)
T = 15 + 273.15 - 6.5 * alt / 1000   # Ambient Temp (K)
Pressure = P0 * np.exp(-M * g *  alt/ (R * T))    # Pressure at Alt (Pa)
Density = Pressure * M / (R * T) # Air Density ()
Density = 1.225 * .84
Throttle = 100.0                 # Throttle value (0 to 100)

# ESC
eCurrMax = 120.0                 # Max current of esc (A      -- Double Check these
eRes = 0.004                     # esc Resistance (milliOhms)      values!!

# Battery Cell -- From LiPos
cells = 6.0                      # Num cells
batVolt = 3.7                    # LiPo per-cell Voltage (V)
batRes = 0.0028                  # bat Resistance (milliOhms)

# Motor
kv = 295.0                       # RPM per Volt 
nLCurr = 1.7                     # No Load Current (A)
nLVolt = 8.4                     # No Load Voltage (V)
mRes = 0.015                     # Motor Resistnace (milliOhms)
mWatt = 2000.0                   # Max Watts of Motor (A)


print('propDiameter: '+str(propDia))
print('altitude: '+str(alt))
print('density: '+str(Density))
print('eCurrMax: '+str(eCurrMax))
print('eRes: '+str(eRes))      
print('battRes: '+str(batRes))           
print('battVolt: '+str(batVolt))
print('cells: '+str(cells))
print('kV: '+str(kv))
print('nLCurr: '+str(nLCurr))
#print('nLVolt: '+str(nLVolt))
print('mRes: '+str(mRes))
print('mWatt: '+str(mWatt))
print('Throttle: '+str(Throttle))

propDiameter: 0.508
altitude: 392.8872
density: 1.0290000000000001
eCurrMax: 120.0
eRes: 0.004
battRes: 0.0028
battVolt: 3.7
cells: 6.0
kV: 295.0
nLCurr: 1.7
mRes: 0.015
mWatt: 2000.0
Throttle: 100.0


In [3]:
""" Pulling Data """

# Set Column Index for Importing Data
header_list = [str(x) for x in range(13)]

# Read file
propData = pd.read_csv(fileName, sep = '\s+', names = header_list)
propData = propData.to_numpy()
df = pd.DataFrame(propData)
df.to_csv(index = False)

# Find All Rows w/ RPM Data
searchString = "RPM"
mask = np.column_stack([df[col].str.contains(r'' + searchString, na = False)
                        for col in df])
RPMdata = pd.DataFrame.dropna(df.loc[mask.any(axis = 1)], axis = 1)

In [4]:
"""
Create New Table to bin RPM Values into Groups
Remove Characters from PROP RPM
Change to Numeric Values for Analysis
"""

# First Find Column w/ Numbers
RPMs = RPMdata.apply(pd.to_numeric, errors = 'ignore')

# Second Remove All non-int64 Columns
RPMs = RPMs.select_dtypes(include = ['int64'])

# Third Get First RPM Value Index
a = RPMs.columns.values
indexinRPM = RPMs[a[0]].index[0]

# Delete Unnecessary Top Rows
RPMdf = df.iloc[indexinRPM:]

# Get Max RPM and Index
maxRPM = RPMs.max()
indexMaxRPM = RPMs[a[0]].idxmax()

# Get Index of Each RPM Value to Bin Data
RPMindex = RPMs.index

# Clear Columns w/ Mostly NA
RPMdf = pd.DataFrame.dropna(RPMdf, axis = 'columns', thresh = 10)

# Add Index of Last Row of Database ot RPM DF to Bin
lastIndex = RPMdf.iloc[[-1]].index.values[0]
# Get Index of RPMs ot Group by RPM
RPMindex = np.append(RPMindex, lastIndex)
RPMlabel = RPMs[a[0]].to_list()

# Label Columns - Assumes consistantcy in value-to-columns, V in first column
Units = RPMdf[RPMdf[0].str.match('V')]

# First Row of "V"
unitsRow = Units[0].index[0]

# Takes First Row, Sets as Column Headers
RPMdf.columns = RPMdf.loc[unitsRow]

# Removes All Non-Numeric Rows
RPMdf = RPMdf.apply(lambda x: pd.to_numeric(x, errors = 'coerce')).dropna()

# Group Each RPM
RPMdf = RPMdf.groupby(pd.cut(RPMdf.index, RPMindex,
                             right = False, labels = RPMlabel))

# Printing Out Grouped DataFrame
for name, group in RPMdf:
    print(name)
    print(group)

1000
9      V     J      Pe      Ct      Cp    PWR  Torque  Thrust
11   0.0  0.00  0.0000  0.0885  0.0330  0.008   0.535   0.451
12   0.5  0.02  0.0633  0.0874  0.0332  0.009   0.539   0.446
13   0.9  0.05  0.1239  0.0862  0.0335  0.009   0.543   0.439
14   1.4  0.07  0.1818  0.0847  0.0336  0.009   0.546   0.432
15   1.8  0.10  0.2368  0.0831  0.0338  0.009   0.549   0.424
16   2.3  0.12  0.2889  0.0814  0.0339  0.009   0.550   0.415
17   2.7  0.14  0.3380  0.0795  0.0340  0.009   0.551   0.405
18   3.2  0.17  0.3841  0.0775  0.0340  0.009   0.551   0.395
19   3.6  0.19  0.4270  0.0753  0.0339  0.009   0.551   0.384
20   4.1  0.22  0.4666  0.0729  0.0339  0.009   0.549   0.372
21   4.6  0.24  0.5030  0.0704  0.0337  0.009   0.547   0.359
22   5.0  0.26  0.5361  0.0678  0.0335  0.009   0.543   0.345
23   5.5  0.29  0.5660  0.0649  0.0331  0.009   0.537   0.331
24   5.9  0.31  0.5926  0.0618  0.0326  0.008   0.530   0.315
25   6.4  0.34  0.6158  0.0586  0.0320  0.008   0.520   0.298
26 

In [43]:
""" Notes & Shortened Terms:
        Curr = Current
        Res = Resistance
        Bat = Battery
        Volt = Voltage
        v = Voltage (when put before term)
        p = Power (when put before term)
        Amps = Amperage
        df = Data Frame
        J = Velocity / (PRs * Diameter (m)) - Prop Advance Ratio
"""

' Notes & Shortened Terms:\n        Curr = Current\n        Res = Resistance\n        Bat = Battery\n        Volt = Voltage\n        v = Voltage (when put before term)\n        p = Power (when put before term)\n        Amps = Amperage\n        df = Data Frame\n        J = Velocity / (PRs * Diameter (m)) - Prop Advance Ratio\n'

In [44]:
""" First Calculations """

RPMgroups = np.array(list(RPMdf.groups))

# allAmps array w/ Expected Amp Draw at Max Static Thrust at each velocity
allAmps = []

for groups in RPMgroups:
    RPMgroup = RPMdf.get_group(groups)
    PWR = RPMgroup['PWR'].iloc[0] * 745.7  # Converting HP to Watts
    allAmps.append(PWR / (cells * batVolt))
    
velocityArray = [0]
velocityStep = 1.524
    
def RPMfinder(motorAmpGuess, batAmpGuess, Output, velocity):
    
    # Main Calc for Iteration of pMotorOut
    def pMotorOutCalc(motorAmp, batAmp):
    
        vBat = cells * batVolt - cells * batRes * batAmp         # Total Bat Volts w/ Volt Loss
        vTerm = (vBat - eRes * batAmp) * Throttle / 100          # Volt Drop from esc/Throttle Val
        vDropMotor = motorAmp * mRes                             # Volt Drop from Motor
        pMotorIn = vTerm * batAmp                                # Power Motor Uses during Opt
        vEMF = vTerm - vDropMotor                                # Voltage Motor after losses
        pMotorOut = vEMF * (motorAmp - nLCurr)                   # Power Motor Generates (Voltage of Motor * (Amps of Motor - No Load Motor Amps))
        pMotorIn = vTerm * (motorAmp - nLCurr)                   # Power Motor Receives

        return pMotorIn, pMotorOut, vTerm, motorAmp, vBat, batAmp, vEMF
    
    pMotorIn, pMotorOut, vTerm, motorAmp, vBat, batAmp, vEMF = pMotorOutCalc(motorAmpGuess, batAmpGuess)
    
    RPM = vEMF * kv                # Revolutions/Min of Motor
    RPS = RPM / 60                 # Revolutions/s of Motor
    
    
    if (batAmp > batVolt/batRes):
        raise Exception("Batteries can't supply this much current: " + str(batAmp))
    if (batAmpGuess > eCurrMax):
        raise Exception("ESC will be overloaded, chose larger ESC: " + str(Amps))
    if (RPM < 0):
        raise Exception("RPM is negative, something is wrong: " + str(RPM))
    
    
    # Find Closest Data to RPM Values, above and below
    RPMgroups = np.array(list(RPMdf.groups))
    
    if (RPM > RPMgroups.max()):
        raise Exception("RPM max limit has been reached")
    if (RPM < RPMgroups.min()):
        raise Exception('RPM minimum limit reached')
    
    bottomRPM = RPMgroups[RPMgroups < RPM].max()
    topRPM = RPMgroups[RPMgroups > RPM].min()
    
    # Get Bottom/Top RPM Groups
    bottomRPMdata = RPMdf.get_group(bottomRPM).reset_index()
    topRPMdata = RPMdf.get_group(topRPM).reset_index()
    
    velocityMPH = velocity * 2.23694      # Conversion to MPH
    
    # Finding Relevant Indices
    bottomRPMClosestVel = abs(bottomRPMdata['V'] - velocityMPH).idxmin()
    topRPMClosestVel = abs(topRPMdata['V'] - velocityMPH).idxmin()
    
    # Finding Ct
    bottomCt = bottomRPMdata['Ct'].iloc[bottomRPMClosestVel]  # Coefficeient of Thrust
    topCt = topRPMdata['Ct'].iloc[topRPMClosestVel]
    
    # Finding Cp
    bottomCp = bottomRPMdata['Cp'].iloc[bottomRPMClosestVel]
    topCp = topRPMdata['Cp'].iloc[topRPMClosestVel]
    
    # Finding Torque
    bottomTorque = bottomRPMdata['Torque'].iloc[bottomRPMClosestVel]
    topTorque = topRPMdata['Torque'].iloc[topRPMClosestVel]
    
    # Interpolate to get Coefficient of Power/Thrust for Motor RPM
    Cp = bottomCp + (RPM - bottomRPM) * (topCp - bottomCp) / (topRPM - bottomRPM)
    Ct = bottomCt + (RPM - bottomRPM) * (topCt - bottomCt) / (topRPM - bottomRPM)
    torque = bottomTorque + (RPM - bottomRPM) * (topTorque - bottomTorque) / (topRPM - bottomRPM)
    
    # Prop Power/Thrust Calc
    pProp = Density * RPS**3 * propDia**5 * Cp         # Power Prop requires in kg * m^2 / s^3 (W)
    ThrustProp = Density * RPS**2 * propDia**4 * Ct    # Thrust Prop generates in kg*m/2  (N)
    
    if (Output == 1):
        print('Export final parameters here')
        
    return pMotorIn, pMotorOut, pProp, vTerm, motorAmp, vBat, batAmp, ThrustProp, vEMF, RPM, torque

In [47]:
""" Final Calculations """

InitMotorGuess = nLCurr + 0.01    # Make First Motor Amp Draw Guess
InitBatGuess = nLCurr + 0.01      # No Load Amp Draw for Motor

# Dataframe for Storing Final Results
resultProp = pd.DataFrame(columns = [ "Battery Current (A)", "RPM (rev/min)", "Thrust (N)", "Power (W)", "Efficiency", "Torque (in-lbf)"])
resultProp.columns.name = 'Velocity (m/s)'

powerTolerance = 0.01 # Tolerance of power to Motor to Prop

velocityPosition = 0 # VelocityArray Position
velocity = velocityArray[velocityPosition] # Current Velocity in m/s

pMotorIn, pMotorOut, pProp, vTerm, motorAmp, vBat, batAmp, ThrustProp, vEMF, RPM, Torque = RPMfinder(InitMotorGuess, InitBatGuess, 0, velocity)

while (ThrustProp > 0):
    
    if (velocity != 0):
        velocityArray.append(velocity)
        
    timeRun = 0   # Counts how many times Loop iterates
    
    pMotorIn, pMotorOut, pProp, vTerm, motorAmp, vBat, batAmp, ThrustProp, vEMF, RPM, Torque = RPMfinder(InitMotorGuess, InitBatGuess, 0, velocity)
    
    
    while (abs(pProp - pMotorOut) > powerTolerance):
        pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBat,batAmp,ThrustProp,vEMF,RPM,Torque = RPMfinder(InitMotorGuess,InitBatGuess,0,velocity)
        
        print('Motor Power: ' + str(pMotorOut) + ' W')
        print('Thrust Prop: ' + str(ThrustProp) + ' N')
        print('Prop Power: ' + str(pProp) + ' W')
        print('pMotorIn: ' + str(pMotorIn) + ' W')
        print('prop calc done\n')
        
        if (pMotorOut < 0):
            raise Exception("Motor Power Out is negative, something is wrong")
        
        pESCOut = pMotorIn
        vESCin = (cells * batVolt)- (cells * batRes * batAmp)
        vESCout = (cells * batVolt) - (cells * batRes + mRes) * batAmp
        InitMotorGuess = pProp / vEMF + nLCurr
        InitBatGuess = pESCOut / vESCout
        
        timeRun = timeRun + 1
        #print("T" + str(timeRun))
        print('Iteration: ' + str(timeRun) + '     Velocity: '+ str(velocityArray[velocityPosition])+' m\s')
    

    
    # Runs one More Time (Could Keep previous motorAmp/BatAmp values, they are close to new)
    pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBat,batAmp,ThrustProp,vEMF,RPM,Torque = RPMfinder(InitMotorGuess,InitBatGuess,0,velocity)
    
    efficiency = pMotorOut / pMotorIn
    
    resultProp.loc[velocityArray[velocityPosition]] = [motorAmp, RPM, ThrustProp, pProp, efficiency, Torque]
    
    velocityPosition += 1
    velocity += velocityStep

thrustLB = resultProp.loc[:, 'Thrust (N)'] * 0.224809  # Newtons to LBS
resultProp['Thrust (lbf)'] = thrustLB

# Removing Last Row b/c FN is 0 so results may be Invalid
resultProp.drop(resultProp.tail(1).index, inplace = True)
resultProp
resultProp.to_csv('TwinMotor.csv')
print("done")

Motor Power: 0.22138782000000023 W
Thrust Prop: 76.15914171619548 N
Prop Power: 2153.984369842795 W
pMotorIn: 0.2216443200000002 W
prop calc done

Iteration: 1     Velocity: 0 m\s
Motor Power: 2015.4456399729297 W
Thrust Prop: 66.6774717267233 N
Prop Power: 1875.7584904488615 W
pMotorIn: 2159.920297386525 W
prop calc done

Iteration: 2     Velocity: 0 m\s
Motor Power: 1701.6844917837686 W
Thrust Prop: 54.582524972196 N
Prop Power: 1397.3690063187958 W
pMotorIn: 1826.9865995014954 W
prop calc done

Iteration: 3     Velocity: 0 m\s
Motor Power: 1418.0158344962197 W
Thrust Prop: 56.26169231018303 N
Prop Power: 1464.9301751490177 W
pMotorIn: 1502.8482677761183 W
prop calc done

Iteration: 4     Velocity: 0 m\s
Motor Power: 1489.5514565773312 W
Thrust Prop: 58.23356389668365 N
Prop Power: 1545.7470137563657 W
pMotorIn: 1580.025215048382 W
prop calc done

Iteration: 5     Velocity: 0 m\s
Motor Power: 1539.3943380493317 W
Thrust Prop: 57.739918333241974 N
Prop Power: 1525.3662533702768 W
pMot

Motor Power: 1146.0811097217613 W
Thrust Prop: 49.093054954365314 N
Prop Power: 1146.054256375813 W
pMotorIn: 1196.3270265607623 W
prop calc done

Iteration: 8     Velocity: 12.192 m\s
Motor Power: 1146.0506565737542 W
Thrust Prop: 49.09265811247846 N
Prop Power: 1146.0420191332237 W
pMotorIn: 1196.2942529336315 W
prop calc done

Iteration: 9     Velocity: 12.192 m\s
Motor Power: 1146.0438626819466 W
Thrust Prop: 47.1384891641075 N
Prop Power: 1115.8312084177644 W
pMotorIn: 1196.2867126634155 W
prop calc done

Iteration: 1     Velocity: 13.716000000000001 m\s
Motor Power: 1117.0840530955738 W
Thrust Prop: 47.27742382895946 N
Prop Power: 1120.1096775577857 W
pMotorIn: 1164.750088749786 W
prop calc done

Iteration: 2     Velocity: 13.716000000000001 m\s
Motor Power: 1121.7807373570963 W
Thrust Prop: 47.4626500109432 N
Prop Power: 1125.8194215784447 W
pMotorIn: 1169.7014842841472 W
prop calc done

Iteration: 3     Velocity: 13.716000000000001 m\s
Motor Power: 1125.5302167290574 W
Thrust P

Motor Power: 768.1138391938679 W
Thrust Prop: 23.588679982796787 N
Prop Power: 766.6027571490672 W
pMotorIn: 789.412347984919 W
prop calc done

Iteration: 4     Velocity: 27.43200000000001 m\s
Motor Power: 766.3246532827452 W
Thrust Prop: 23.55069578847952 N
Prop Power: 765.3007009847532 W
pMotorIn: 787.5412894674616 W
prop calc done

Iteration: 5     Velocity: 27.43200000000001 m\s
Motor Power: 765.3777323044396 W
Thrust Prop: 23.561227357139376 N
Prop Power: 765.6616741333859 W
pMotorIn: 786.538958759093 W
prop calc done

Iteration: 6     Velocity: 27.43200000000001 m\s
Motor Power: 765.6934599530575 W
Thrust Prop: 23.565572257638692 N
Prop Power: 765.8106054539684 W
pMotorIn: 786.870042716274 W
prop calc done

Iteration: 7     Velocity: 27.43200000000001 m\s
Motor Power: 765.7976930377936 W
Thrust Prop: 23.56380743251543 N
Prop Power: 765.7501114808676 W
pMotorIn: 786.9806126918007 W
prop calc done

Iteration: 8     Velocity: 27.43200000000001 m\s
Motor Power: 765.7467914043707 W
Th

Motor Power: 145.3660899281101 W
Thrust Prop: 0.0 N
Prop Power: 145.7041888112291 W
pMotorIn: 146.1964414299428 W
prop calc done

Iteration: 2     Velocity: 41.14800000000002 m\s
Motor Power: 145.96332254091763 W
Thrust Prop: 0.0 N
Prop Power: 146.43279935469033 W
pMotorIn: 146.79714642588732 W
prop calc done

Iteration: 3     Velocity: 41.14800000000002 m\s
Motor Power: 146.42938457346386 W
Thrust Prop: 0.0 N
Prop Power: 146.42321426177529 W
pMotorIn: 147.26803340223944 W
prop calc done

Iteration: 4     Velocity: 41.14800000000002 m\s
done


In [48]:
path = "C:/Users/marym/OneDrive/Documents/PropellerCalc/"
batdir = path + "batteries"
propdir = path + "propellors"
motordir = path + "motors"

In [ ]:
for battery in os.listdir(batdir):
    bat = os.path.join(directory, battery)
    
    for prop in os.listdir(propdir):
        prop = os.path.join(directory, prop)
        
        for motor in os.listdir(motordir):
            motor = os.path.join(directory, motor)
            ## Put code in from above